In [1]:
# Our Study is performed using GridSearchCV. Sample code of our study is shown on CRM_ALL_Part2_Sample.ipynb 
#This notebook is preperad to reproduce 10 StratifiedKFold for Single candidate on CMS-ALL Dataset

In [2]:
import pickle
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [3]:
x = pickle.load( open( "CMS_ALL_Dataset_x.p", "rb" ) )
y = pickle.load( open( "CMS_ALL_Dataset_y.p", "rb" ) )

In [4]:
def eval(model, data, target,nkf, score = lambda y,yp: np.mean(y==yp) ):
  
    v_accuracy_score = 0
    v_precision_macro_score=0
    v_recall_macro_score=0
    v_f1_macro_score=0
    v_precision_weighted_score=0
    v_recall_weighted_score=0
    v_f1_weighted_score=0

    # 10 StratifiedKFold with shuffling
    kf =StratifiedKFold(n_splits=nkf,shuffle=True)
    for ind_train, ind_test in kf.split(data,target):
        
        #Model Train
        model.fit(data[ind_train],target[ind_train])
        
        #Model Test Prediction
        ypred = model.predict(data[ind_test])

        #Model Accuracy Test
        v_accuracy_score += score(target[ind_test],ypred)

        #Model Precision,Recall,F1 Score with Macro Avarage Test
        v_precision_macro_score+=precision_score(target[ind_test], ypred, labels=[1,2,3,4], average='macro')
        v_recall_macro_score+=recall_score(target[ind_test], ypred, labels=[1,2,3,4], average='macro')
        v_f1_macro_score+=f1_score(target[ind_test], ypred, labels=[1,2,3,4], average='macro')

        #Model Precision,Recall,F1 Score with Weighted Avarage Test
        v_precision_weighted_score+=precision_score(target[ind_test], ypred, labels=[1,2,3,4], average='weighted')
        v_recall_weighted_score+=recall_score(target[ind_test], ypred, labels=[1,2,3,4], average='weighted')
        v_f1_weighted_score+=f1_score(target[ind_test], ypred, labels=[1,2,3,4], average='weighted')

    # Mean of Test scores
    v_accuracy_score=v_accuracy_score/nkf
    v_precision_macro_score=v_precision_macro_score/nkf
    v_recall_macro_score=v_recall_macro_score/nkf
    v_f1_macro_score=v_f1_macro_score/nkf
    v_precision_weighted_score=v_precision_weighted_score/nkf
    v_recall_weighted_score=v_recall_weighted_score/nkf
    v_f1_weighted_score=v_f1_weighted_score/nkf
    
    return (v_accuracy_score,v_precision_macro_score,v_recall_macro_score,v_f1_macro_score,v_precision_weighted_score,v_recall_weighted_score,v_f1_weighted_score,model)

In [5]:
# Hyper Parameters(Hidden layers & Number of neoruns in each layer, Alpha,Activation function) For CMS-ALL. 

iHiddenLayers=(1000,500)
iAlpha=0.0001
iActivationFunction='logistic'



In [6]:
# MLPClassifier parameters are setted.
clf = MLPClassifier(hidden_layer_sizes=iHiddenLayers,max_iter=500,alpha=iAlpha,activation=iActivationFunction)

# 10-StratifiedKFold for Candidate iHiddenLayers,iAlpha,iActivationFunction
result=eval(clf, x,y,10)

# Print Results
dash = '-' * 125
print(dash)
print(' {:<30s}  {:<10}  {:<10}  {:<10s}  {:<10}  {:<10s}  {:<10s}  {:<10s}'.format('Parameters(Hidden Layers','Accuracy','Precision','Recall','F1 Score','Precision','Recall','F1 Score'))
print(' {:<30s}  {:<10}  {:<10}  {:<10s}  {:<10}  {:<10s}  {:<10s}  {:<10s}'.format(',alpha,Activation func.)','','Macro','Macro','Macro','Weighted','Weighted','Weighted'))
print(dash)
print(' {:<30s}  {:<12.3f} {:<10.3f} {:<11.3f} {:<11.3} {:<12.3f} {:<12.3f} {:<10.3f}'.format(str(result[7].hidden_layer_sizes)+' '+str(result[7].alpha)+' '+result[7].activation,result[0],result[1],result[2],result[3],result[4],result[5],result[6]))



-----------------------------------------------------------------------------------------------------------------------------
 Parameters(Hidden Layers        Accuracy    Precision   Recall      F1 Score    Precision   Recall      F1 Score  
 ,alpha,Activation func.)                    Macro       Macro       Macro       Weighted    Weighted    Weighted  
-----------------------------------------------------------------------------------------------------------------------------
 (1000, 500) 0.0001 logistic     0.864        0.808      0.728       0.746       0.871        0.864        0.860     


In [7]:
# ANN Model
ANNModel=result[7]
ANNModel

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000, 500), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)